In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import numpy as np
import ast

import iqplot


import bokeh.io
import bokeh.models
import bokeh.plotting

import holoviews as hv

hv.extension("bokeh")

bokeh.io.output_notebook()

In [ ]:
Data = pd.read_csv(
    "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas_Agar/Results/DataSetNew.csv")

In [ ]:

#Data = Data[Data['Test Starvation'] == "Overnight no Water"]
#Data = Data[Data["Training Starvation"] == "Not starved"]#.reset_index()
# Prep dataset from data
Melted = pd.melt(
    Data,
    id_vars=["Training", "ObjectsReinforced", "Reinforced_side", "Date", "Fly"],
    value_name="Durations",
    value_vars=[
        "Durations Left Corner",
        "Durations Right Corner",
        "Durations Top Corner",
        "Durations Left Front",
        "Durations Right Front",
        "Durations Top Front",
    ],
    var_name="Variable",
)

conditions = [
    (
        Melted["Reinforced_side"].str.contains("Right")
        & (Melted["Variable"].str.contains("Right"))
    ),
    (
        Melted["Reinforced_side"].str.contains("Left")
        & (Melted["Variable"].str.contains("Left"))
    ),
    (
        Melted["Reinforced_side"].str.contains("Right")
        & (Melted["Variable"].str.contains("Left"))
    ),
    (
        Melted["Reinforced_side"].str.contains("Left")
        & (Melted["Variable"].str.contains("Right"))
    ),
    (Melted["Variable"].str.contains("Top")),
]

values = [
    "Rewarded",
    "Rewarded",
    "Punished",
    "Punished",
    "Empty",
]
Melted["Condition"] = np.select(conditions, values)

conditions = [
    ((Melted["Variable"].str.contains("Corner"))),
    ((Melted["Variable"].str.contains("Front"))),
]
values = [
    "Corner",
    "Front",
]
Melted["Location"] = np.select(conditions, values)

Melted = Melted.loc[Melted.Location=="Corner"]

In [ ]:
Treshs = list(range(20, 300, 10))


In [ ]:
CI_Data = pd.DataFrame


#for t in Treshs:
def TreshComp (Treshs):
    for t in Treshs:
        for index, row in Melted.iterrows():
        # print(row['Durations Left Corner'])

        # print (1 for i in row['Durations Left Corner'])
            Melted.loc[index, "Peeks"] = sum(
                1 for i in ast.literal_eval(row["Durations"]) if i > t
            )
        Stat_Data=Melted.groupby(['Training', 'Condition'])
        Groups = list(Stat_Data.groups.keys())

        ConfInts = Stat_Data['Peeks'].apply(lambda x: draw_bs_ci(x))
        Means = Stat_Data['Peeks'].mean()

        C_Unst = ConfInts.unstack().reset_index()
        CMelt = pd.melt(
            C_Unst,
            id_vars=["Training",],
            value_name="CI",
            value_vars=[
                "Empty",
                "Punished",
                "Rewarded"
            ],
            var_name="Condition",
        )

        M_Unst = Means.unstack().reset_index()

        MMelt = pd.melt(
            M_Unst,
            id_vars=["Training",],
            value_name="Mean",
            value_vars=[
                "Empty",
                "Punished",
                "Rewarded"
            ],
            var_name="Condition",
        )

        Merged= pd.merge(CMelt, MMelt, on=["Training", "Condition"])

        Merged['Treshold'] = t

        yield(Merged)
        #CI_Data= pd.concat(CI_Data,Merged)

In [ ]:
CI_Data = pd.concat(TreshComp(Treshs))


In [ ]:
Rewardeds = CI_Data.loc[CI_Data.Condition == "Rewarded"]
RewardedsC = Rewardeds.loc[Rewardeds.Training == "Ctrl"]

RewardedsC['LowerBounds'] = RewardedsC.CI.str[0]
RewardedsC['UpperBounds'] = RewardedsC.CI.str[1]

RewardedsT = Rewardeds.loc[Rewardeds.Training == "Trained"]

RewardedsT['LowerBounds'] = RewardedsT.CI.str[0]
RewardedsT['UpperBounds'] = RewardedsT.CI.str[1]


In [ ]:
Punishes = CI_Data.loc[CI_Data.Condition == "Punished"]
PunishesC = Punishes.loc[Punishes.Training == "Ctrl"]

PunishesC['LowerBounds'] = PunishesC.CI.str[0]
PunishesC['UpperBounds'] = PunishesC.CI.str[1]

PunishesT = Punishes.loc[Punishes.Training == "Trained"]

PunishesT['LowerBounds'] = PunishesT.CI.str[0]
PunishesT['UpperBounds'] = PunishesT.CI.str[1]


In [ ]:
x = 'Peeking Threshold'
y = 'Count'


# Make the plot
s1 = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title='Empty chamber peekings',
)

s1.circle(
    source=EmptysC,
    x='Treshold',
    y='Mean',
)

s1.segment(
    source=EmptysC,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    line_width=2
)

s1.circle(
    source=EmptysT,
    x='Treshold',
    y='Mean',
    color='orange',
)

s1.segment(
    source=EmptysT,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    color='orange',
    line_width=2
)


###
s2 = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title='Rewarded chamber peekings',

)

s2.circle(
    source=RewardedsC,
    x='Treshold',
    y='Mean',
)

s2.segment(
    source=RewardedsC,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    line_width=2
)

s2.circle(
    source=RewardedsT,
    x='Treshold',
    y='Mean',
    color='orange',
)

s2.segment(
    source=RewardedsT,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    color='orange',
    line_width=2
)

###
s3 = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title='Punished chamber peekings',

)

s3.circle(
    source=PunishesC,
    x='Treshold',
    y='Mean',
)

s3.segment(
    source=PunishesC,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    line_width=2
)

s3.circle(
    source=PunishesT,
    x='Treshold',
    y='Mean',
    color='orange',
)

s3.segment(
    source=PunishesT,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    color='orange',
    line_width=2
)
#p = bokeh.io.hplot(s1,s2)


bokeh.io.show(bokeh.layouts.column(s1,s2,s3))


In [ ]:
Groups[1][1]

In [ ]:
rg = np.random.default_rng()

def draw_bs_rep(data, func, rg):
    """Compute a bootstrap replicate from data."""
    bs_sample = rg.choice(data, size=len(data))
    return func(bs_sample)

def draw_bs_ci(data, func=np.mean, rg=rg, n_reps=2000):
    """Sample bootstrap multiple times and compute confidence interval"""
    bs_reps = np.array(
        [draw_bs_rep(data, func, rg) for _ in range(n_reps)]
    )
    conf_int = np.percentile(bs_reps,[2.5, 97.5])
    return(conf_int)

Stat_Data=Melted.groupby(['Training', 'Condition'])
Stat_Data

Groups = list(Stat_Data.groups.keys())

Treshs = list(range(20, 300, 10))

CI_Data = pd.DataFrame


#for t in Treshs:
def TreshComp (Treshs):
    for t in Treshs:
        for index, row in Melted.iterrows():
        # print(row['Durations Left Corner'])

        # print (1 for i in row['Durations Left Corner'])
            Melted.loc[index, "Peeks"] = sum(
                1 for i in ast.literal_eval(row["Durations"]) if i > t
            )
        Stat_Data=Melted.groupby(['Training', 'Condition'])
        Groups = list(Stat_Data.groups.keys())

        ConfInts = Stat_Data['Peeks'].apply(lambda x: draw_bs_ci(x))
        Means = Stat_Data['Peeks'].mean()

        C_Unst = ConfInts.unstack().reset_index()
        CMelt = pd.melt(
            C_Unst,
            id_vars=["Training",],
            value_name="CI",
            value_vars=[
                "Empty",
                "Punished",
                "Rewarded"
            ],
            var_name="Condition",
        )

        M_Unst = Means.unstack().reset_index()

        MMelt = pd.melt(
            M_Unst,
            id_vars=["Training",],
            value_name="Mean",
            value_vars=[
                "Empty",
                "Punished",
                "Rewarded"
            ],
            var_name="Condition",
        )

        Merged= pd.merge(CMelt, MMelt, on=["Training", "Condition"])

        Merged['Treshold'] = t

        yield(Merged)

CI_Data = pd.concat(TreshComp(Treshs))

CI_Data["LowerBounds"] = CI_Data.CI.str[0]
CI_Data["UpperBounds"] = CI_Data.CI.str[1]
CI_Data["LowerErr"] = CI_Data["Mean"]-CI_Data["LowerBounds"]
CI_Data["UpperErr"] = CI_Data["UpperBounds"] - CI_Data["Mean"]

Emptys = CI_Data.loc[CI_Data.Condition == "Empty"]
EmptysC = Emptys.loc[Emptys.Training == "Ctrl"]
EmptysT = Emptys.loc[Emptys.Training == "Trained"]

Rewardeds = CI_Data.loc[CI_Data.Condition == "Rewarded"]
RewardedsC = Rewardeds.loc[Rewardeds.Training == "Ctrl"]
RewardedsT = Rewardeds.loc[Rewardeds.Training == "Trained"]

Punishes = CI_Data.loc[CI_Data.Condition == "Punished"]
PunishesC = Punishes.loc[Punishes.Training == "Ctrl"]
PunishesT = Punishes.loc[Punishes.Training == "Trained"]

x = 'Peeking Threshold'
y = 'Count'

# Make the plot
s1 = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title='Empty chamber peekings',
)

s1.circle(
    source=EmptysC,
    x='Treshold',
    y='Mean',
)

s1.segment(
    source=EmptysC,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    line_width=2
)

s1.circle(
    source=EmptysT,
    x='Treshold',
    y='Mean',
    color='orange',
)

s1.segment(
    source=EmptysT,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    color='orange',
    line_width=2
)


###
s2 = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title='Rewarded chamber peekings',

)

s2.circle(
    source=RewardedsC,
    x='Treshold',
    y='Mean',
)

s2.segment(
    source=RewardedsC,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    line_width=2
)

s2.circle(
    source=RewardedsT,
    x='Treshold',
    y='Mean',
    color='orange',
)

s2.segment(
    source=RewardedsT,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    color='orange',
    line_width=2
)

###
s3 = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title='Punished chamber peekings',

)

s3.circle(
    source=PunishesC,
    x='Treshold',
    y='Mean',
)

s3.segment(
    source=PunishesC,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    line_width=2
)

s3.circle(
    source=PunishesT,
    x='Treshold',
    y='Mean',
    color='orange',
)

s3.segment(
    source=PunishesT,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    color='orange',
    line_width=2
)
#p = bokeh.io.hplot(s1,s2)


bokeh.io.show(bokeh.layouts.column(s1,s2,s3))

In [ ]:
Plotest = hv.Scatter(
    Emptys,
    kdims=['Treshold'],
    vdims=['Mean', 'Training'],
).opts(
    color='Training',
    cmap=['blue', 'orange']
) * hv.ErrorBars(Emptys,
                       kdims=['Treshold'],
                       vdims=['Mean', 'LowerErr', 'UpperErr', 'Training']).opts(
    upper_head=None,
    lower_head=None,
    color='Training',

)



Plotest

In [ ]:
Subset = Melted.loc[Melted["ObjectsReinforced"] == 'Orange']

def ThePlot (Data) :
    rg = np.random.default_rng()

    def draw_bs_rep(data, func, rg):
        """Compute a bootstrap replicate from data."""
        bs_sample = rg.choice(data, size=len(data))
        return func(bs_sample)

    def draw_bs_ci(data, func=np.mean, rg=rg, n_reps=2000):
        """Sample bootstrap multiple times and compute confidence interval"""
        bs_reps = np.array(
            [draw_bs_rep(data, func, rg) for _ in range(n_reps)]
        )
        conf_int = np.percentile(bs_reps,[2.5, 97.5])
        return(conf_int)

    Stat_Data=Data.groupby(['Training', 'Condition'])

    Groups = list(Stat_Data.groups.keys())

    Treshs = list(range(20, 300, 10))

    def TreshComp (Treshs):
        for t in Treshs:
            for index, row in Data.iterrows():
            # print(row['Durations Left Corner'])

            # print (1 for i in row['Durations Left Corner'])
                Data.loc[index, "Peeks"] = sum(
                    1 for i in ast.literal_eval(row["Durations"]) if i > t
                )
            Stat_Data=Data.groupby(['Training', 'Condition'])
            Groups = list(Stat_Data.groups.keys())

            ConfInts = Stat_Data['Peeks'].apply(lambda x: draw_bs_ci(x))
            Means = Stat_Data['Peeks'].mean()

            C_Unst = ConfInts.unstack().reset_index()
            CMelt = pd.melt(
                C_Unst,
                id_vars=["Training",],
                value_name="CI",
                value_vars=[
                    "Empty",
                    "Punished",
                    "Rewarded"
                ],
                var_name="Condition",
            )

            M_Unst = Means.unstack().reset_index()

            MMelt = pd.melt(
                M_Unst,
                id_vars=["Training",],
                value_name="Mean",
                value_vars=[
                    "Empty",
                    "Punished",
                    "Rewarded"
                ],
                var_name="Condition",
            )

            Merged= pd.merge(CMelt, MMelt, on=["Training", "Condition"])

            Merged['Treshold'] = t

            yield(Merged)

    CI_Data = pd.concat(TreshComp(Treshs))

    CI_Data["LowerBounds"] = CI_Data.CI.str[0]
    CI_Data["UpperBounds"] = CI_Data.CI.str[1]
    CI_Data["LowerErr"] = CI_Data["Mean"]-CI_Data["LowerBounds"]
    CI_Data["UpperErr"] = CI_Data["UpperBounds"] - CI_Data["Mean"]

    Emptys = CI_Data.loc[CI_Data.Condition == "Empty"]
    EmptysC = Emptys.loc[Emptys.Training == "Ctrl"]
    EmptysT = Emptys.loc[Emptys.Training == "Trained"]

    Rewardeds = CI_Data.loc[CI_Data.Condition == "Rewarded"]
    RewardedsC = Rewardeds.loc[Rewardeds.Training == "Ctrl"]
    RewardedsT = Rewardeds.loc[Rewardeds.Training == "Trained"]

    Punishes = CI_Data.loc[CI_Data.Condition == "Punished"]
    PunishesC = Punishes.loc[Punishes.Training == "Ctrl"]
    PunishesT = Punishes.loc[Punishes.Training == "Trained"]

    x = 'Peeking Threshold'
    y = 'Count'


    # Make the plot
    s1 = bokeh.plotting.figure(
        height=300,
        width=450,
        x_axis_label=x,
        y_axis_label=y,
        title='Empty chamber peekings',
    )

    s1.circle(
        source=EmptysC,
        x='Treshold',
        y='Mean',
    )

    s1.segment(
        source=EmptysC,
        x0='Treshold',
        y0='LowerBounds',
        x1='Treshold',
        y1='UpperBounds',
        line_width=2
    )

    s1.circle(
        source=EmptysT,
        x='Treshold',
        y='Mean',
        color='orange',
    )

    s1.segment(
        source=EmptysT,
        x0='Treshold',
        y0='LowerBounds',
        x1='Treshold',
        y1='UpperBounds',
        color='orange',
        line_width=2
    )


    ###
    s2 = bokeh.plotting.figure(
        height=300,
        width=450,
        x_axis_label=x,
        y_axis_label=y,
        title='Rewarded chamber peekings',

    )

    s2.circle(
        source=RewardedsC,
        x='Treshold',
        y='Mean',
    )

    s2.segment(
        source=RewardedsC,
        x0='Treshold',
        y0='LowerBounds',
        x1='Treshold',
        y1='UpperBounds',
        line_width=2
    )

    s2.circle(
        source=RewardedsT,
        x='Treshold',
        y='Mean',
        color='orange',
    )

    s2.segment(
        source=RewardedsT,
        x0='Treshold',
        y0='LowerBounds',
        x1='Treshold',
        y1='UpperBounds',
        color='orange',
        line_width=2
    )

    ###
    s3 = bokeh.plotting.figure(
        height=300,
        width=450,
        x_axis_label=x,
        y_axis_label=y,
        title='Punished chamber peekings',

    )

    s3.circle(
        source=PunishesC,
        x='Treshold',
        y='Mean',
    )

    s3.segment(
        source=PunishesC,
        x0='Treshold',
        y0='LowerBounds',
        x1='Treshold',
        y1='UpperBounds',
        line_width=2
    )

    s3.circle(
        source=PunishesT,
        x='Treshold',
        y='Mean',
        color='orange',
    )

    s3.segment(
        source=PunishesT,
        x0='Treshold',
        y0='LowerBounds',
        x1='Treshold',
        y1='UpperBounds',
        color='orange',
        line_width=2
    )
    return s1, s2, s3

#s1, s2, s3 = ThePlot(Subset)

#Lay = bokeh.layouts.column(s1,s2,s3)
#bokeh.io.show(bokeh.layouts.column(s1,s2,s3))
#bokeh.io.save(Lay,title="Orange object rewarded" ,filename= "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Plots/BS_CIs/OrangeReinforced.html" )

In [ ]:
Subset = Melted
s1, s2, s3 = ThePlot(Subset)
Lay = bokeh.layouts.column(s1,s2,s3)
bokeh.io.save(Lay,title="Full Experiment" ,filename= "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas_Agar/Results/BS_CI/Full.html" )

In [ ]:
Subset = Melted
s1, s2, s3 = ThePlot(Subset)
Lay = bokeh.layouts.column(s1,s2,s3)
bokeh.io.save(Lay,title="Full Experiment" ,filename= "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Plots/BS_CIs/Full.html" )

In [ ]:
Subset = Melted.loc[Melted['Date']== '22-08-04']

s1, s2, s3 = ThePlot(Subset)
bokeh.io.show(bokeh.layouts.column(s1,s2,s3))

#bokeh.io.save(bokeh.layouts.column(s1,s2,s3), #filename="/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data#/MultiSensory_Project/GatedArenas/Results/22-08-04_Plot.png")

In [ ]:
Subset = Melted.loc[Melted['Date']== '22-07-28']

s1, s2, s3 = ThePlot(Subset)
bokeh.io.show(bokeh.layouts.column(s1,s2,s3))

In [ ]:
Subset = Melted.loc[Melted['Date']== '22-07-29']

s1, s2, s3 = ThePlot(Subset)
bokeh.io.show(bokeh.layouts.column(s1,s2,s3))

In [ ]:
Subset = Melted.loc[Melted['Date']== '22-08-04']

s1, s2, s3 = ThePlot(Subset)
bokeh.io.show(bokeh.layouts.column(s1,s2,s3))

In [ ]:
s1 = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title='Empty chamber peekings',
)

s1.circle(
    source=EmptysC,
    x='Treshold',
    y='Mean',
)

s1.segment(
    source=EmptysC,
    x0='Treshold',
    y0='LowerBounds',
    x1='Treshold',
    y1='UpperBounds',
    line_width=2
)

def Canvas (Title, x, y):
    canvas = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title=Title,
    )
    return canvas

def BS_CI (canvas, data, x, y, low_y, high_y):
    canvas.circle(
    source=data,
    x=x,
    y=y,
    )

    canvas.segment(
    source=data,
    x0=x,
    y0=low_y,
    x1=x,
    y1=high_y,
    line_width=2
    )
    return canvas




In [ ]:
# Functions definition

rg = np.random.default_rng()
def draw_bs_rep(data, func, rg):
    """Compute a bootstrap replicate from data."""
    bs_sample = rg.choice(data, size=len(data))
    return func(bs_sample)

def draw_bs_ci(data, func=np.mean, rg=rg, n_reps=2000):
    """Sample bootstrap multiple times and compute confidence interval"""
    bs_reps = np.array(
        [draw_bs_rep(data, func, rg) for _ in range(n_reps)]
    )
    conf_int = np.percentile(bs_reps,[2.5, 97.5])
    return(conf_int)


def TreshComp (Treshs):
    for t in Treshs:
        for index, row in Melted.iterrows():
        # print(row['Durations Left Corner'])

        # print (1 for i in row['Durations Left Corner'])
            Melted.loc[index, "Peeks"] = sum(
                1 for i in ast.literal_eval(row["Durations"]) if i > t
            )
        Stat_Data=Melted.groupby(['Training', 'Condition'])
        Groups = list(Stat_Data.groups.keys())

        ConfInts = Stat_Data['Peeks'].apply(lambda x: draw_bs_ci(x))
        Means = Stat_Data['Peeks'].mean()

        C_Unst = ConfInts.unstack().reset_index()
        CMelt = pd.melt(
            C_Unst,
            id_vars=["Training",],
            value_name="CI",
            value_vars=[
                "Empty",
                "Punished",
                "Rewarded"
            ],
            var_name="Condition",
        )

        M_Unst = Means.unstack().reset_index()

        MMelt = pd.melt(
            M_Unst,
            id_vars=["Training",],
            value_name="Mean",
            value_vars=[
                "Empty",
                "Punished",
                "Rewarded"
            ],
            var_name="Condition",
        )

        Merged= pd.merge(CMelt, MMelt, on=["Training", "Condition"])

        Merged['Treshold'] = t

        yield(Merged)

def Canvas (Title, x, y):
    canvas = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label=x,
    y_axis_label=y,
    title=Title,
    )
    return canvas

def BS_CI (canvas, data, x, y, low_y, high_y, color):
    canvas.circle(
    source=data,
    x=x,
    y=y,
    color=color,
    )

    canvas.segment(
    source=data,
    x0=x,
    y0=low_y,
    x1=x,
    y1=high_y,
    line_width=2,
    color=color
    )
    return canvas


In [ ]:
Subset = Melted.loc[Melted["ObjectsReinforced"] == 'Orange']

def ThePlot (Data) :

    Stat_Data=Data.groupby(['Training', 'Condition'])

    Groups = list(Stat_Data.groups.keys())

    Treshs = list(range(20, 300, 10))

    CI_Data = pd.concat(TreshComp(Treshs))

    CI_Data["LowerBounds"] = CI_Data.CI.str[0]
    CI_Data["UpperBounds"] = CI_Data.CI.str[1]
    CI_Data["LowerErr"] = CI_Data["Mean"]-CI_Data["LowerBounds"]
    CI_Data["UpperErr"] = CI_Data["UpperBounds"] - CI_Data["Mean"]

    Emptys = CI_Data.loc[CI_Data.Condition == "Empty"]
    EmptysC = Emptys.loc[Emptys.Training == "Ctrl"]
    EmptysT = Emptys.loc[Emptys.Training == "Trained"]

    Rewardeds = CI_Data.loc[CI_Data.Condition == "Rewarded"]
    RewardedsC = Rewardeds.loc[Rewardeds.Training == "Ctrl"]
    RewardedsT = Rewardeds.loc[Rewardeds.Training == "Trained"]

    Punishes = CI_Data.loc[CI_Data.Condition == "Punished"]
    PunishesC = Punishes.loc[Punishes.Training == "Ctrl"]
    PunishesT = Punishes.loc[Punishes.Training == "Trained"]

    x = 'Peeking Threshold'
    y = 'Count'


    # Make the plot
    s1 = Canvas('Empty chamber peekings', x, y)
    s1 = BS_CI(s1, EmptysC, 'Treshold', 'Mean', 'LowerBounds', 'UpperBounds', 'blue')
    s1 = BS_CI(s1, EmptysT, 'Treshold', 'Mean', 'LowerBounds', 'UpperBounds', 'orange')

    s2 = Canvas('Rewarded chamber peekings', x, y)
    s2 = BS_CI(s2, RewardedsC, 'Treshold', 'Mean', 'LowerBounds', 'UpperBounds', 'blue')
    s2 = BS_CI(s2, RewardedsT, 'Treshold', 'Mean', 'LowerBounds', 'UpperBounds', 'orange')

    s3 = Canvas('Punished chamber peekings', x, y)
    s3 = BS_CI(s3, PunishesC, 'Treshold', 'Mean', 'LowerBounds', 'UpperBounds', 'blue')
    s3 = BS_CI(s3, PunishesT, 'Treshold', 'Mean', 'LowerBounds', 'UpperBounds', 'orange')

    return s1, s2, s3


In [ ]:
Subset = Melted.loc[Melted["ObjectsReinforced"] == 'Orange']
s1, s2, s3 = ThePlot(Subset)
bokeh.io.show(bokeh.layouts.column(s1,s2,s3))

In [ ]:
Subset = Melted.loc[Melted["ObjectsReinforced"] == 'Blue']
s1, s2, s3 = ThePlot(Subset)
#bokeh.io.show(bokeh.layouts.column(s1,s2,s3))

In [ ]:
Subset = Melted
s1, s2, s3 = ThePlot(Subset)
bokeh.io.show(bokeh.layouts.column(s1,s2,s3))